In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.datasets import load_iris
import numpy as np

X, y = load_iris(return_X_y=True)
print(X.shape, y.shape) 

(150, 4) (150,)


In [16]:
n = X.shape[0]
rnd = np.arange(n)
np.random.seed(42)
np.random.shuffle(rnd)

In [19]:
Xs = X[rnd]
ys = y[rnd]

In [26]:
X_train,X_test = Xs[:105],Xs[105:]

In [27]:
Y_train,Y_test = ys[:105],ys[105:]

feature Independance ~ means and variances for every class of every feature , No shared Covariance

Means[k][f] - mean of every kth class in fth feature 
variance[k][f] - variance of -----
probabilities for every kth class - log(prior[k]) - 1/2 (log(2*pie*variance[k][j]) + (xj-u[k][j])**2/variance[k][j])
             class having heighest prob will we predicted

In [28]:
means = np.zeros((np.unique(y).shape[0],X.shape[1]))

In [29]:
k = np.unique(y).shape[0]

In [41]:
for i in range(k):
    for j in range(X.shape[1]):
        means[i][j] = X_train[Y_train==i,j].mean()
        
    

In [42]:
variances = np.array([X_train[Y_train==i].var(axis=0) for i in np.unique(Y_train)])

In [44]:
variances

array([[0.1165    , 0.12219375, 0.0316    , 0.012     ],
       [0.2332966 , 0.11702479, 0.2368595 , 0.04707071],
       [0.41      , 0.13616211, 0.34631836, 0.07652344]])

In [45]:
priors = np.bincount(Y_train)/Y_train.shape[0]

In [46]:
priors

array([0.38095238, 0.31428571, 0.3047619 ])

In [73]:
probs = np.zeros((k,X_test.shape[0]))

In [74]:
probs.shape

(3, 45)

In [86]:
for i in range(k):
    prob = np.sum((X_test -means[i])**2/variances[i] + np.log(6.28*variances[i]),axis=1)
    p = np.log(priors[i]) - 0.5*prob
    probs[i] = p
    

In [79]:
probs

array([[ 2.29889356e+00, -1.45379839e+02, -1.69039534e+02,
         8.43702662e+00, -1.00141467e+02, -1.04033407e+02,
        -1.18410410e+02, -1.05231686e+02, -1.49650250e+02,
        -4.81664410e-01, -1.44635932e+02, -9.59475581e+01,
        -1.35178784e+02, -6.15431614e+01, -8.28022055e+01,
        -9.89683745e+01,  8.81140513e+00, -9.65384443e+01,
        -8.32586320e+01,  6.51431203e-01, -8.47443927e+01,
        -1.47274804e+02, -1.49342770e+02, -6.82353303e+00,
        -5.41267556e+01, -1.66009437e+02, -1.31010080e+02,
         8.52630078e+00, -1.59442119e+02,  6.81475150e+00,
        -1.11724539e+02, -1.48296912e+02, -1.31907009e+02,
        -8.43552048e+01, -1.31592286e+02, -8.96312163e+01,
        -9.09332398e+01, -1.25750915e+02, -1.22688064e+02,
        -1.71475162e+00, -8.44896645e+01, -1.02689950e+02,
         2.02478521e+00, -7.82170717e+01, -1.52996507e+02],
       [ 1.62532113e+01, -1.45278685e+01, -2.00424559e+01,
         1.81667390e+01, -3.47598527e+00, -4.75713935e+

In [71]:
(np.sum((X_test - means[0])/variances[0] +  np.log(6.28*variances[0]),axis=1)).shape

(45,)

In [77]:
(np.log(priors[i]) - 0.5*(np.sum((X_test - means[0])/variances[0] +  np.log(6.28*variances[0]),axis=1))).shape

(45,)

In [82]:
probs.shape

(3, 45)

In [87]:
max_p = np.argmax(probs,axis=0)

In [88]:
max_p

array([0, 2, 2, 0, 1, 1, 2, 1, 2, 0, 2, 1, 2, 1, 1, 1, 0, 1, 1, 0, 1, 2,
       2, 0, 1, 2, 2, 0, 2, 0, 2, 2, 2, 1, 2, 1, 1, 2, 2, 0, 1, 1, 0, 1,
       2], dtype=int64)

In [85]:
Y_test

array([0, 2, 2, 0, 1, 1, 2, 1, 2, 0, 2, 1, 2, 1, 1, 1, 0, 1, 1, 0, 1, 2,
       2, 0, 1, 2, 2, 0, 2, 0, 1, 2, 2, 1, 2, 1, 1, 2, 2, 0, 1, 2, 0, 1,
       2])

In [90]:
from sklearn.metrics import classification_report

In [91]:
print(classification_report(Y_test,max_p))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.94      0.94      0.94        17
           2       0.94      0.94      0.94        18

    accuracy                           0.96        45
   macro avg       0.96      0.96      0.96        45
weighted avg       0.96      0.96      0.96        45



In [96]:
class Gaussian_Naive_Bayes:
    def __init__(self,fit_prior=True):
        self.fit_prior = fit_prior
    def fit(self,X,y):
        self.k = np.unique(y).shape[0]
        self.means = np.array([X[y==i].mean(axis=0) for i in range(self.k)])
        self.variances = np.array([X[y==i].var(axis=0) for i in range(self.k)])
        self.priors = np.bincount(y)/y.shape[0]
    def predict(self,X):
        probs = np.zeros((self.k,X.shape[0]))
        for i in range(self.k):
            prob = np.sum((X -self.means[i])**2/self.variances[i] + np.log(6.28*self.variances[i]),axis=1)
            p = np.log(self.priors[i]) - 0.5*prob
            probs[i] = p
        preds = np.argmax(probs,axis=0)
        return preds

In [97]:
gnb = Gaussian_Naive_Bayes()

In [98]:
gnb.fit(X_train,Y_train)

In [100]:
print(classification_report(Y_train,gnb.predict(X_train)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       0.91      0.94      0.93        33
           2       0.94      0.91      0.92        32

    accuracy                           0.95       105
   macro avg       0.95      0.95      0.95       105
weighted avg       0.95      0.95      0.95       105



In [101]:
print(classification_report(Y_test,gnb.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.94      0.94      0.94        17
           2       0.94      0.94      0.94        18

    accuracy                           0.96        45
   macro avg       0.96      0.96      0.96        45
weighted avg       0.96      0.96      0.96        45



In [102]:
from sklearn.datasets import load_wine

In [103]:
wine = load_wine()

In [118]:
X2n = wine.data
y2n = wine.target

In [119]:
n = X2n.shape[0]
rnd = np.arange(n)
np.random.seed(42)
np.random.shuffle(rnd)

In [120]:
X2 = X2n[rnd]
y2 = y2n[rnd]

In [121]:
X.shape

(178, 13)

In [122]:
X_train2,X_test2 = X2[:140],X2[140:]
Y_train2,Y_test2 = y2[:140],y2[140:]

In [123]:
gnb2 = Gaussian_Naive_Bayes()

In [124]:
gnb2.fit(X_train2,Y_train2)

In [125]:
print(classification_report(Y_train2,gnb2.predict(X_train2)))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        48
           1       0.98      0.98      0.98        56
           2       0.97      1.00      0.99        36

    accuracy                           0.99       140
   macro avg       0.99      0.99      0.99       140
weighted avg       0.99      0.99      0.99       140



In [126]:
print(classification_report(Y_test2,gnb2.predict(X_test2)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      1.00      1.00        15
           2       1.00      1.00      1.00        12

    accuracy                           1.00        38
   macro avg       1.00      1.00      1.00        38
weighted avg       1.00      1.00      1.00        38



lets compare with Scikit-learn

In [127]:
from sklearn.naive_bayes import GaussianNB

In [128]:
gnbr = GaussianNB()

In [129]:
gnbr.fit(X_train2,Y_train2)

GaussianNB()

In [130]:
print(classification_report(Y_train2,gnbr.predict(X_train2)))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        48
           1       0.98      0.98      0.98        56
           2       0.97      1.00      0.99        36

    accuracy                           0.99       140
   macro avg       0.99      0.99      0.99       140
weighted avg       0.99      0.99      0.99       140



In [131]:
print(classification_report(Y_test2,gnbr.predict(X_test2)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      1.00      1.00        15
           2       1.00      1.00      1.00        12

    accuracy                           1.00        38
   macro avg       1.00      1.00      1.00        38
weighted avg       1.00      1.00      1.00        38

